Copyright (c) 2025 Mitsuru Ohno  
Use of this source code is governed by a BSD-3-style  
license that can be found in the LICENSE file.  

## 使用方法の要約
基本的な使用: RxnIVPsolv("sample_data.csv")でインスタンス化  
数値積分: get_ode_system()で必要なオブジェクトを取得  
エラーハンドリング: try-except文でエラーをキャッチ  
可視化: matplotlibで結果をプロット  
パラメータ解析: 異なる初期条件での比較  
これらの使用例は、現在のコードが正しく動作することを前提としています。もしエラーが発生した場合は、debug_ode_system()メソッドで詳細な情報を確認できます。  

## 反応式を記載したcsvファイルを指定する  

In [ ]:
file_path = '../sample_data/sample_data_1b.csv'  # CSVファイルのパスを指定

In [ ]:
# development phase
# Add the parent directory (one level up from the notebook's location) to the Python path
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..'))

## 数値積分を実行する場合  

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
# from rxnfit.build_ode import RxnODEbuild
from src.rxnfit.build_ode import RxnODEbuild

In [ ]:
# 作成した微分方程式に関する情報を表示
def get_ode_info(rxn_ivp, debug_info=False):    
    print(f"number of species: {len(rxn_ivp.function_names)}")
    print(f"unique species: {rxn_ivp.function_names}")
    print(f"rate constant: {rxn_ivp.rate_consts_dict}")

    if debug_info is True:
        # デバッグ情報を確認
        print("\n=== debug info ===")
        debug_info = rxn_ivp.debug_ode_system()
        print(f"order of args: {debug_info['lambdify_args']}")
        print(f"system of ODE: {debug_info['ode_expressions']}")

In [ ]:
@dataclass
class SolverConfig:
    rxn_ivp: RxnODEbuild
    y0: list              # 初期濃度（必須）
    t_span: tuple         # 時間範囲（必須）
    t_eval: Optional[np.ndarray] = field(default=None)  # 任意
    method: str = "RK45"  # 任意
    rtol: float = 1e-6    # 任意

In [ ]:
def solve_system(config: SolverConfig):

    # 数値積分に必要なオブジェクトを取得
    ode_construct = rxn_ivp_build.get_ode_system()
    (system_of_equations, sympy_symbol_dict, 
     ode_system, function_names, rate_consts_dict) = ode_construct
    
    # 微分方程式の右辺を定義
    def system_rhs(t, y):
        """ODEシステムの右辺を計算する関数"""
        rhs_odesys = []
        for i, species_name in enumerate(function_names):
            if species_name in ode_system:
                try:
                    rhs_odesys.append(ode_system[species_name](t, *y))
                except Exception as e:
                    print(f"Error in {species_name}: {e}")
                    rhs_odesys.append(0.0)
            else:
                rhs_odesys.append(0.0)
        return rhs_odesys
    
    # 数値積分を実行
    #print("\n=== 数値積分の実行 ===")
    solution = None
    try:
        solution = solve_ivp(
            system_rhs, 
            config.t_span, 
            config.y0, 
            t_eval=config.t_eval,
            method='RK45'
        )
        #print("数値積分が成功しました！")
    except Exception as e:
        print(f"数値積分でエラーが発生しました: {e}")
        print("デバッグ情報を確認してください。")
    return ode_construct, solution


In [ ]:
# 結果をプロット
def solution_plot(function_names, solution):
    print("\n=== 結果のプロット ===")
    plt.figure(figsize=(12, 8))
    
    for i, species_name in enumerate(function_names):
        plt.plot(solution.t, solution.y[i], label=species_name, linewidth=2)
    
    plt.xlabel('Time (s)', fontsize=12)
    plt.ylabel('Concentration', fontsize=12)
    plt.title('Chemical Reaction Kinetics - Sample Data', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 最終時刻での濃度を表示
    print("\n=== 最終時刻での濃度 ===")
    final_concentrations = {name: conc[-1] for name, conc in zip(function_names, solution.y)}
    for name, conc in final_concentrations.items():
        print(f"{name}: {conc:.6f}")
        


In [ ]:
# 基本的な数値積分 -インスタンス化-
rxn_ivp_build = RxnODEbuild(file_path)

In [ ]:
# 使い方例
rxn_ivp_build = RxnODEbuild(file_path)
get_ode_info(rxn_ivp_build, debug_info=True)

In [ ]:
# 初期値等の入力項目を渡す
config = SolverConfig(
    rxn_ivp=rxn_ivp_build, 
    y0=[1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    t_span=(0, 10)
)

In [ ]:
rxn_ivp_build.rate_consts_dict['k1']+3

In [ ]:
intedrated_result = solve_system(config)
intedrated_result

In [ ]:
solution_plot(rxn_ivp_build.function_names,intedrated_result[1])

## フィッティングさせる  

In [ ]:
file_path = '../sample_data/sample_time_course_2.csv'  # CSVファイルのパスを指定

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv(file_path)
l = ['t', 'AcOEt', 'H2O', 'AcOH', 'EtOH']
df = df[l]
df

In [ ]:
rxn_ivp_build.function_names

n系列への対応例

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

# ---- 任意の ODE ----
def reaction_ode(t, C, k):
    return -k * C  # 例：1次反応

# ---- ODE を解く関数 ----
def solve_concentration(t_data, C0, k):
    sol = solve_ivp(
        fun=lambda t, C: reaction_ode(t, C, k),
        t_span=(t_data[0], t_data[-1]),
        y0=[C0],
        t_eval=t_data
    )
    return sol.y[0]

# ---- n 系列対応の目的関数 ----
def objective(k, datasets):
    k = k[0]
    total_error = 0.0

    for t_data, C_obs in datasets:
        C0 = C_obs[0]
        C_pred = solve_concentration(t_data, C0, k)
        total_error += np.sum((C_pred - C_obs)**2)

    return total_error

# ---- 例：n 系列のデータをまとめる ----
datasets = [
    (np.array([0, 1, 2, 3, 4]), np.array([1.0, 0.7, 0.5, 0.35, 0.25])),
    (np.array([0, 1, 2, 3, 4]), np.array([2.0, 1.4, 1.0, 0.7, 0.5])),
    (np.array([0, 1, 2, 3, 4]), np.array([0.5, 0.35, 0.25, 0.18, 0.13])),
    # 必要なら何系列でも追加できる
]

# ---- 最適化 ----
initial_guess = [0.5]

result = minimize(
    objective,
    initial_guess,
    args=(datasets,),
    bounds=[(0, None)]
)

k_opt = result.x[0]
print("最適化された k =", k_opt)


In [ ]:
# ２系列用
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize

# ---- 例としての ODE ----
def reaction_ode(t, C, k):
    return -k * C

# ---- ODE を解く関数 ----
def solve_concentration(t_data, C0, k):
    sol = solve_ivp(
        fun=lambda t, C: reaction_ode(t, C, k),
        t_span=(t_data[0], t_data[-1]),
        y0=[C0],
        t_eval=t_data
    )
    return sol.y[0]

# ---- 誤差関数（目的関数） ----
def objective(k, t1, C1_obs, C0_1, t2, C2_obs, C0_2):
    k = k[0]
    C1_pred = solve_concentration(t1, C0_1, k)
    C2_pred = solve_concentration(t2, C0_2, k)

    # 二系列の残差平方和を合計
    error = np.sum((C1_pred - C1_obs)**2) + np.sum((C2_pred - C2_obs)**2)
    return error

# ---- 実験データ（例） ----
t1 = np.array([0, 1, 2, 3, 4])
C1_obs = np.array([1.0, 0.7, 0.5, 0.35, 0.25])
C0_1 = C1_obs[0]

t2 = np.array([0, 1, 2, 3, 4])
C2_obs = np.array([2.0, 1.4, 1.0, 0.7, 0.5])
C0_2 = C2_obs[0]

# ---- 最適化 ----
initial_guess = [0.5]

result = minimize(
    objective,
    initial_guess,
    args=(t1, C1_obs, C0_1, t2, C2_obs, C0_2),
    bounds=[(0, None)]
)

k_opt = result.x[0]
print("最適化された k =", k_opt)


## 実験データを読み込んでscipyでフィットさせる

In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

In [11]:
import numpy as np
import pandas as pd
np.random.seed(1)  # 再現性のため

from scipy.integrate import solve_ivp
from scipy.optimize import minimize

from src.rxnfit.expdata_fit import time_course, expdata_read

In [12]:


# 時間軸（0〜10 の 11 点）
t = np.linspace(0, 10, 11)

# 仮想的な濃度変化（適当な指数関数で生成）
A = np.exp(-0.3 * t)                # A は減少
B = 0.8 * (1 - np.exp(-0.3 * t)) * np.exp(-0.1 * t)  # B は一度増えて減る
P = 1 - A - B                       # P は生成物として増える

# --- 欠損値を入れる ---
A[[2, 5]] = np.nan   # A の t=2,5 を欠損に
P[[3, 8]] = np.nan   # P の t=3,8 を欠損に

# DataFrame にまとめる
df = pd.DataFrame({
    "time": t,
    "A": A,
    "B": B,
    "P": P
})

print(df)


    time         A         B         P
0    0.0  1.000000  0.000000  0.000000
1    1.0  0.740818  0.187614  0.071568
2    2.0       NaN  0.295521  0.155667
3    3.0  0.406570  0.351699       NaN
4    4.0  0.301194  0.374739  0.324067
5    5.0       NaN  0.376956  0.399914
6    6.0  0.165299  0.366475  0.468226
7    7.0  0.122456  0.348620  0.528923
8    8.0  0.090718  0.326853       NaN
9    9.0  0.067206  0.303397  0.629398
10  10.0  0.049787  0.279651  0.670562


In [13]:
# 時間軸（0〜12 の 13 点）
t = np.linspace(0, 12, 13)

# 仮想的な濃度変化
A = np.exp(-0.25 * t)                         # A は減少
B = 0.9 * (1 - np.exp(-0.25 * t)) * np.exp(-0.15 * t)  # B は山型
P = 1 - A - B                                 # P は増加

# 欠損をランダムに入れる
A[[3, 9]] = np.nan
B[[5]] = np.nan
P[[2, 7, 11]] = np.nan

# DataFrame にまとめる
df2 = pd.DataFrame({
    "time": t,
    "A": A,
    "B": B,
    "P": P
})

print(df2)


    time         A         B         P
0    0.0  1.000000  0.000000  0.000000
1    1.0  0.778801  0.171349  0.049850
2    2.0  0.606531  0.262340       NaN
3    3.0       NaN  0.302791  0.224843
4    4.0  0.367879  0.312224  0.319897
5    5.0  0.286505       NaN  0.410167
6    6.0  0.223130  0.284267  0.492603
7    7.0  0.173774  0.260215       NaN
8    8.0  0.135335  0.234389  0.630276
9    9.0       NaN  0.208725  0.685876
10  10.0  0.082085  0.184333  0.733582
11  11.0  0.063928  0.161795       NaN
12  12.0  0.049787  0.141362  0.808851


In [7]:
# 時間軸（0〜12 の 13 点）
t = np.linspace(0, 12, 13)

# 仮想的な濃度変化
A = np.exp(-0.25 * t)                         # A は減少
B = 0.9 * (1 - np.exp(-0.25 * t)) * np.exp(-0.15 * t)  # B は山型

# 欠損をランダムに入れる
A[[3, 9]] = np.nan
B[[5]] = np.nan

# DataFrame にまとめる
df3 = pd.DataFrame({
    "time": t,
    "A": A,
    "B": B,
})

print(df3)

    time         A         B
0    0.0  1.000000  0.000000
1    1.0  0.778801  0.171349
2    2.0  0.606531  0.262340
3    3.0       NaN  0.302791
4    4.0  0.367879  0.312224
5    5.0  0.286505       NaN
6    6.0  0.223130  0.284267
7    7.0  0.173774  0.260215
8    8.0  0.135335  0.234389
9    9.0       NaN  0.208725
10  10.0  0.082085  0.184333
11  11.0  0.063928  0.161795
12  12.0  0.049787  0.141362


In [5]:
time_course(df)

([array([ 0.,  1.,  3.,  4.,  6.,  7.,  8.,  9., 10.]),
  array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
  array([ 0.,  1.,  2.,  4.,  5.,  6.,  7.,  9., 10.])],
 [array([1.        , 0.74081822, 0.40656966, 0.30119421, 0.16529889,
         0.12245643, 0.09071795, 0.06720551, 0.04978707]),
  array([0.        , 0.1876139 , 0.29552143, 0.35169921, 0.37473882,
         0.3769563 , 0.36647495, 0.34862019, 0.32685341, 0.30339675,
         0.27965104]),
  array([0.        , 0.07156788, 0.15566693, 0.32406697, 0.39991354,
         0.46822617, 0.52892338, 0.62939774, 0.67056189])])

In [9]:
expdata_read([df, df2])

[    time         A         B         P
 0    0.0  1.000000  0.000000  0.000000
 1    1.0  0.740818  0.187614  0.071568
 2    2.0       NaN  0.295521  0.155667
 3    3.0  0.406570  0.351699       NaN
 4    4.0  0.301194  0.374739  0.324067
 5    5.0       NaN  0.376956  0.399914
 6    6.0  0.165299  0.366475  0.468226
 7    7.0  0.122456  0.348620  0.528923
 8    8.0  0.090718  0.326853       NaN
 9    9.0  0.067206  0.303397  0.629398
 10  10.0  0.049787  0.279651  0.670562,
     time         A         B         P
 0    0.0  1.000000  0.000000  0.000000
 1    1.0  0.778801  0.171349  0.049850
 2    2.0  0.606531  0.262340       NaN
 3    3.0       NaN  0.302791  0.224843
 4    4.0  0.367879  0.312224  0.319897
 5    5.0  0.286505       NaN  0.410167
 6    6.0  0.223130  0.284267  0.492603
 7    7.0  0.173774  0.260215       NaN
 8    8.0  0.135335  0.234389  0.630276
 9    9.0       NaN  0.208725  0.685876
 10  10.0  0.082085  0.184333  0.733582
 11  11.0  0.063928  0.161795       NaN

In [10]:
popt, pcov = curve_fit(model, t_all, y_exp, p0=[1.0, 1.0, 1.0])

NameError: name 'curve_fit' is not defined